# 在Notebook中使用Bokeh绘制图表

Bokeh是Continuum Analytics开发的一套基于Python和Javascript的绘图库。它可以在Notebook中使用，官方推荐的方式是：

```python
from bokeh.io import output_notebook
output_notebook()
```

它缺省从CDN载入Javascript和CSS库。但是在没有网络连接的环境中无法使用Bokeh。这时可以使用:

```python
from bokeh.resources import INLINE
output_notebook(INLINE)
```

使用`INLINE`将所有的Javascript和CSS库嵌入到Notebook之中，虽然它可以离线使用，但是会增大Notebook体积。这里介绍一种新的方法。

首先调用`copy_bokeh_resources()`将资源文件复制到`static`文件夹之下：

In [8]:
def copy_bokeh_resources():
    import os
    import shutil
    from os import path
    from bokeh.resources import INLINE

    for folder in ["static/js", "static/css"]:
        if not path.exists(folder):
            os.makedirs(folder)

    for fn in INLINE._file_paths("js"):
        shutil.copy(fn, "static/js")
    for fn in INLINE._file_paths("css"):
        shutil.copy(fn, "static/css")

In [9]:
copy_bokeh_resources()

然后使用`Resource`对象指定资源载入的方式和URL：

In [10]:
from bokeh.io import output_notebook
from bokeh.resources import Resources

output_notebook(Resources(mode="server", root_url="./"))

Loading BokehJS ...

In [12]:
from bokeh.plotting import figure
from bokeh.io import show
import numpy as np

x = np.linspace(0, 4*np.pi, 500)
y = np.sin(x)

fig = figure(height=300)
fig.line(x, y)
show(fig);

在Bokeh 0.12中，创建的所有`Figure`对象会被累积到当前文档中。例如运行上述程序多次之后，会发现当前文档中包含多个`Figure`对象。这个BUG会导致程序运行越来越慢，内存越占越多。

In [13]:
from bokeh.plotting import curdoc
doc = curdoc()
doc.roots

为了解决上述问题，可以在每次调用`show()`显示图表之前，用`curdoc().clear()`清空当前文档。

In [14]:
curdoc().clear()
doc.roots

[]

Bokeh绘制的图表在Python和Javascript分别有对应的对象。在Python修改图表中对象的属性之后，调用`push_notebook()`将修改信息发送给Javascript刷新图表中的显示。下面使用`line`变量保存`fig.line()`的返回对象，然后调用`show()`显示一副图表：

In [15]:
fig = figure(height=300)
line = fig.line(x, y)
show(fig);

下面修改`line.glyph.line_width`属性，它表示曲线的宽度，然后调用`push_notebook()`刷新图表显示：

In [16]:
from bokeh.io import push_notebook
line.glyph.line_width = 2
push_notebook()

In [17]:
curdoc().clear()

## 使用`push_notebook()`刷新多幅图表

使用前面所述的方法，只能更新最后显示的图表，本节介绍刷新多幅图表的方法。

调用`figure()`创建图表时，会创建相应的`Document`对象，而`push_notebook()`则将该`Document`对象中被修改的内容发送到Javascript刷新显示。如果直接创建`Figure()`对象，则不会创建相应的`Document`对象。

In [18]:
from bokeh.plotting import Figure
from bokeh.document import Document
from bokeh.io import curstate

def show_figure(fig):
    if fig.document is None:
        doc = Document()
        doc.add_root(fig)
        curstate().document = doc
    fig.__dict__["_handle"] = show(fig)
    
def update_figure(fig):
    push_notebook(fig.document, None, fig._handle)

In [19]:
fig1 = Figure(plot_height=300)
line1 = fig1.line(x, y)
show_figure(fig1);

In [20]:
fig2 = Figure(plot_height=300)
line2 = fig2.line(x, y**2)
show_figure(fig2);

下面调用通过设置`line1.data_source.data["y"]`修改曲线的Y轴数据。通过`line2.glyph.line_color`修改曲线的颜色，然后调用`update_figure()`更新图表。

In [21]:
line1.data_source.data["y"] = np.sin(x) / x
line2.glyph.line_color = "red"
update_figure(fig1)
update_figure(fig2)

## 用ipywidgets制作图表控制面板

我们可以使用ipywidgets库创建控制图表的控件，在控件的事件响应函数中修改图表的各个对象的属性，然后调用`update_figure()`更新图表。下面是一个例子。

程序中，为了方便更新曲线数据，❶先创建一个`ColumnDataSource`对象，❷在调用`fig.line()`时，使用`source`参数指定曲线的数据源为该对象。使用这种方法绘制曲线时，`line()`的头两个参数为X轴和Y轴数据在`ColumnDataSource`对象中对应的键。

❸为滑块控件`freq_slider`绑定回调函数`callback`，当滑块的`value`属性值被修改时，会调用`callback`函数。其参数`event`中保存与时间相关的信息，`event["new"]`为滑块的新位置。

In [22]:
from ipywidgets import FloatSlider
from IPython.display import display
from bokeh.models import ColumnDataSource

fig = Figure(plot_height=300)
data = ColumnDataSource(data=dict(x=[], y=[]), id="data") #❶
fig.line("x", "y", source=data, line_width=2)  #❷

def update_source(source, freq):
    x = np.linspace(0, 2, 100)
    y = np.sin(2*np.pi*freq*x)
    source.data["x"] = x
    source.data["y"] = y

def callback(event):
    freq = event["new"]
    update_source(data, freq)
    update_figure(fig)

freq_slider = FloatSlider(value=0, min=0, max=3, description="Freq:")
freq_slider.observe(callback, names=["value"])  #❸
display(freq_slider)
show_figure(fig)
freq_slider.value = 1.0